In [ ]:
!wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-02 07:22:43--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-02 07:22:43--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc30b35efc61fea634608d0b79f.dl.dropboxusercontent.com/cd/0/inline/BNsxwSSAdu8LffwLqNmtnKQD3ThGz8tz7R9k3DLRiVAyGewbBxtmNhk6mVlO-4dWYY4FJplT7mvymhg1SmGFWDsmVzkUBxjwMJc-gR_6OLmjGdaI8uniKzDtRF6kl_7_QXXLGTK0g3PCN3nHS5WESjef/file# [following]
--2021-05-02 07:22:43--  https://ucc30b35efc61fea634608d0b79f.dl.dropboxusercontent.com/cd/0/inline/BNsxwSSAdu8LffwLqNmtnKQD3ThGz8tz7R9k3DLRiVAyGewbBxtmNhk6mVlO-

In [ ]:
!wget https://www.dropbox.com/sh/cestloicrvvi69s/AADjx7RXR5eM64XMVKt0NTxRa?dl=0 -O upvote_model.zip
!unzip -o upvote_model.zip -d upvote_model

--2021-05-02 07:24:23--  https://www.dropbox.com/sh/cestloicrvvi69s/AADjx7RXR5eM64XMVKt0NTxRa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6032:18::a27d:5212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/cestloicrvvi69s/AADjx7RXR5eM64XMVKt0NTxRa [following]
--2021-05-02 07:24:24--  https://www.dropbox.com/sh/raw/cestloicrvvi69s/AADjx7RXR5eM64XMVKt0NTxRa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc95388832b2c1412332d2f3fef2.dl.dropboxusercontent.com/zip_download_get/AxEAGWsYZDTQWIaN-8MMdRr1f2A8TNHDHxlXGYHhKI1DS2owIr10bHuj9XNw1a7HpZpxjtepL7ppGWgvSyszCPGl2LEy7LZxS_KUY9VaTfC9RA [following]
--2021-05-02 07:24:24--  https://uc95388832b2c1412332d2f3fef2.dl.dropboxusercontent.com/zip_download_get/AxEAGWsYZDTQWIaN-8MMdRr1f2A8TNHDHxlXGYHhKI1DS2owIr10bHuj9XNw1a7HpZpxjtepL7ppGW

In [ ]:
!nvidia-smi

Sun May  2 07:24:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pickle

import torch
!pip install transformers
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
from scipy import stats
import collections
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
with open('/content/gdrive/My Drive/file.txt', 'w') as f:
  f.write('content')

In [ ]:
# load sample data
# format { post_id: [post, [comment1, comment2, ... ] }

data_file_path = 'grouped_data.pickle'

def load_data(file_path):
    posts_comments = []
    upvotes = []
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        for post_id, val in data.items():
            post = val[0]
            comms = val[1]
            for com in comms:
                try:
                    if len(post['title'] + " " + post['selftext']) < 500 and len(com['body']) < 500:
                        posts_comments.append((post['title'] + " " + post['selftext'], com['body']))
                        upvotes.append(com['score'])
                except:
                    pass
                
    return posts_comments, upvotes
    
posts_comms, upvotes = load_data(data_file_path)

In [ ]:
upvotes = np.array(upvotes)
q1 = np.quantile(upvotes, 0.6)
q2 = np.quantile(upvotes, 0.80)
q3 = np.quantile(upvotes, 0.95)

classes = []
for label in upvotes:
  if label < q1:
    classes.append(0)
  elif label < q2:
    classes.append(1)
  elif label < q3:
    classes.append(2)
  else:
    classes.append(3)

collections.Counter(classes)

Counter({0: 558219, 1: 297786, 2: 176570, 3: 54782})

In [ ]:
# split text
train_texts, test_texts, train_labels, test_labels = train_test_split(posts_comms, classes, test_size=.01, train_size=.2, shuffle=True)
# train_texts, test_texts, train_labels, test_labels = train_test_split(posts_comms, upvotes, test_size=.1, train_size=.2, shuffle=True)
print(len(train_texts), len(train_labels), len(test_texts), len(test_labels))

posts_comms = None

217471 217471 10874 10874


In [ ]:
# load tokenizer
tokenizer = BertTokenizerFast.from_pretrained('distilbert-base-cased', use_cache=True)

In [ ]:
# tokenize train data

train_post = []
train_comm = []

for x in tqdm(train_texts):
    train_post.append(x[0])
    train_comm.append(x[1])

train_texts = None
train_encodings = tokenizer(text=train_post, text_pair=train_comm, truncation=True, padding=True)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# tokenize test data

test_post = []
test_comm = []

for x in tqdm(test_texts):
    test_post.append(x[0])
    test_comm.append(x[1])

test_texts = None

test_encodings = tokenizer(text=test_post, text_pair=test_comm, truncation=True, padding=True)


In [ ]:
# load model
model = BertForSequenceClassification.from_pretrained('upvote_model', num_labels=4)
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# create dataset class and load encodings and associated labels to it

class upvote_prediction_dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = upvote_prediction_dataset(train_encodings, train_labels)
test_dataset = upvote_prediction_dataset(test_encodings, test_labels)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

trainer.save_model()

Step,Training Loss
10,1.084200
20,1.095000
30,1.057700
40,1.154500
50,1.041800
60,1.127400
70,1.032000
80,1.184400
90,1.136600
100,1.016100


In [ ]:

trainer.save_model('/content/gdrive/My Drive/model')